<img src="https://www.uni-flensburg.de/fileadmin/content/system/logos/europa/uni-flensburg.png" alt="EUF" height="200" width="300"  align="right" >
<img src="https://hs-flensburg.de/themes/hsfl/img/Logo_color_schrift_grau_72dpi-320px.png" alt="HSF" height="200" width="200"  align="left" >


In [ ]:
__copyright__ = "Zentrum für nachhaltige Energiesysteme Flensburg"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "wolfbunke, ulfmueller"


# <center> OpenMod Workshop Zürich 2018</center>

## Open source tools for cross-grid-level electricity grid optimization developed in the open_eGo project - Learn-a-thon

****

### Learn more about


* __[open_eGo Project Webpage](https://openegoproject.wordpress.com/)__
* __[eTraGo Tool Documentation](http://etrago.readthedocs.io/en/latest/index.html)__  
*  __[oedb Tutorials](http://oep.iks.cs.ovgu.de/dataedit/)__  How to use the OpenEnergy Database
* __[OpenMod Forum](https://forum.openmod-initiative.org/t/learn-a-thon-using-tools-for-cross-grid-level-electricity-grid-optimization-developed-in-the-open-ego-project/856)__ 

<img src="http://etrago.readthedocs.io/en/latest/_images/etrago_logo.png" alt="HSF" height="200" width="200"  align="right" >
## Table of Contents <a class="anchor" id="toc"></a>


* [Getting started with eTraGo](#started)
* [LOPF Calculation of Germany and neighbours with 10 notes](#d-kmean10)
* [LOPF Calculation of Schleswig-Holstein](#shcalc)
* [Using snapshot clustering](#snapshot)

# Getting started with eTraGo <a class="anchor" id="started"></a>


The python package eTraGo provides a optimization of flexibility options for transmission grids based on PyPSA. A speciality in this context is that transmission grids are described by the 380, 220 and 110 kV in Germany. Conventionally the 110kV grid is part of the distribution grid. The integration of the transmission and ‘upper’ distribution grid is part of eTraGo.

The focus of optimization are flexibility options with a special focus on energy storages and grid expansion measures.




## Installation 

Please, find more information on the [README.md](https://github.com/openego/eGo/tree/features/tutorial/ego/examples/tutorials#etrago). 


## Import eTraGo packages

We are importing the [main function](https://github.com/openego/eTraGo/blob/dev/etrago/appl.py) of eTraGo and its database and plotting functions. 


In [ ]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
np.random.seed()
import time
import oedialect

import os

if not 'READTHEDOCS' in os.environ:
    # Sphinx does not run this code.
    # Do not import internal packages directly  
    from etrago.tools.io import NetworkScenario, results_to_oedb
    from etrago.tools.plot import (plot_line_loading, plot_stacked_gen,
                                     add_coordinates, curtailment, gen_dist,
                                     storage_distribution,storage_expansion)
    from etrago.tools.utilities import (load_shedding, data_manipulation_sh,
                                    results_to_csv, parallelisation, pf_post_lopf,  
                                    loading_minimization, calc_line_losses, group_parallel_lines)
    from etrago.cluster.networkclustering import busmap_from_psql, cluster_on_extra_high_voltage, kmean_clustering
    from egoio.tools import db
    from sqlalchemy.orm import sessionmaker
    from etrago.appl import etrago
    
    

In [ ]:
# enable jupyter interactiv plotting
%matplotlib notebook
from ipywidgets import *
import matplotlib.pyplot as plt

# LOPF Calculation of Germany and neighbours with 30 nodes <a class="anchor" id="d-kmean10"></a>


<div style="text-align: right"> 
[top](#toc) </div>

In this section we start our eTraGo calulation with an __Status Quo__ scenario of Germany an its electrical neighbours. For time and performents reasons we are useing the [k-mean](https://de.wikipedia.org/wiki/K-Means-Algorithmus) clustering [functionality of eTraGo](http://etrago.readthedocs.io/en/latest/api/etrago.cluster.html#etrago.cluster.networkclustering.kmean_clustering) and use $k=30$ nodes. For the same reason we choose the time period of __start_snapshot__ and __end_snapshot__ for a day with 24 hours of the scenario year.  


### Make your calulation settings

A detailed discription of the args python dictionary can be found under <http://etrago.readthedocs.io/en/latest/_modules/etrago/appl.html#etrago>. 




In [ ]:
args_k10 = {# Setup and Configuration:
        'db': 'oedb',                # db session
        'gridversion': "v0.2.11",         # None for model_draft or Version number (e.g. v0.2.11) for grid schema
        'method': 'lopf',            # lopf or pf
        'pf_post_lopf': False,       # state whether you want to perform a pf after a lopf simulation
        'start_snapshot': 4393,      # 2.07.
        'end_snapshot' : 4400,
        'scn_name': 'NEP 2035',    # state which scenario you want to run: Status Quo, NEP 2035, eGo100
        'solver': 'glpk',            # glpk, cplex or gurobi
        # Export options:
        'lpfile': False,             # state if and where you want to save pyomo's lp file: False or /path/tofolder
        'results': False,            # state if and where you want to save results as csv: False or /path/tofolder
        'export': False,             # state if you want to export the results back to the database
        # Settings:        
        'storage_extendable':True,   # state if you want storages to be installed at each node if necessary.
        'generator_noise':True,      # state if you want to apply a small generator noise 
        'reproduce_noise': False,    # state if you want to use a predefined set of random noise for the given scenario. 
                                     # if so, provide path, e.g. 'noise_values.csv'
        'minimize_loading':False,
        # Clustering:
        'k_mean_clustering': 30,     # state if you want to perform a k-means clustering on the given network. 
                                     # State False or the value k (e.g. 20).
        'network_clustering': False, # state if you want to perform a clustering of HV buses to EHV buses.
        # Simplifications:
        'parallelisation':False,     # state if you want to run snapshots parallely.
        'skip_snapshots':False,
        'line_grouping': False,      # state if you want to group lines running between the same buses.
        'branch_capacity_factor': 0.7, # globally extend or lower branch capacities
        'load_shedding':False,       # meet the demand at very high cost; for debugging purposes.
        'comments':None }

In [ ]:
# Start eTraGo calculation with args setting
# create network object which incluedes all input and output data
network = etrago(args_k10)

In [ ]:
# plot generation distripution
gen_dist(network, techs=None, snapshot=1,n_cols=3,gen_size=0.02)

In [ ]:
# plot stacked sum of nominal power for each generator type and timestep
#fig.set_size_inches(14,14)
# fix error in .../eGo/ego/examples/tutorials/src/etrago/etrago/tools/plot.py
# 'wind_offshore':'skyblue', wind_onshore':'skyblue',
plot_stacked_gen(network, resolution="MW")

In [ ]:
# make a line loading plot
# basemade einbauen für hintergrund länder
fig,ax = plt.subplots(1,1)
fig.set_size_inches(8,8)
plot_line_loading(network)

In [ ]:
# plot to show extendable storages, if expanded
storage_expansion(network)

In [ ]:
# get storage sizes in MW
network.storage_units.p_nom_opt.groupby(network.storage_units.carrier, axis=0).sum()

### Marginal price per bus node

In [ ]:
# get snapshot
now = network.snapshots[2]

fig,ax = plt.subplots(1,1)
fig.set_size_inches(6,4)

network.plot(ax=ax,line_widths=pd.Series(0.5,network.lines.index))
plt.hexbin(network.buses.x, network.buses.y, 
           gridsize=20,
           C=network.buses_t.marginal_price.loc[now],
           cmap=plt.cm.jet)

#for some reason the colorbar only works with graphs plt.plot
#and must be attached plt.colorbar

cb = plt.colorbar()
cb.set_label('Locational Marginal Price (EUR/MWh)') 

### Change the scnario 

* Set <b style="color:Tomato;">'scn_name':</b> to  'NEP 2035' and recalculate. 



****




In [ ]:
# delete eTraGo object
#del network

# LOPF Calculation of Schleswig-Holstein <a class="anchor" id="shcalc"></a>


<div style="text-align: right"> 
[top](#toc) </div>


### Case Schleswig-Holstein

The data set of Schleswing-Holstein is an extract of the main data set and works as an island. The power production and flows of the  adjacent network areas are neglected. Therefore, the installed capacity and power production is very high.

For our analysis we used serveral plotting options of eTraGo of [etrago.tools.plot](http://etrago.readthedocs.io/en/latest/api/etrago.tools.html#module-etrago.tools.plot).


### Make your settings


In [ ]:
args_sh = {# Setup and Configuration:
        'db': 'oedb',                # db session
        'gridversion': "v0.2.11",         # None for model_draft or Version number (e.g. v0.2.11) for grid schema
        'method': 'lopf',            # lopf or pf
        'pf_post_lopf': False,       # state whether you want to perform a pf after a lopf simulation
        'start_snapshot': 4393,      # 6552
        'end_snapshot' : 4394,
        'scn_name': 'SH NEP 2035',   # state which scenario you want to run: Status Quo, NEP 2035, eGo100
        'solver': 'glpk',            # glpk, cplex or gurobi
        # Export options:
        'lpfile': False,             # state if and where you want to save pyomo's lp file: False or /path/tofolder
        'results': False,            # state if and where you want to save results as csv: False or /path/tofolder
        'export': False,             # state if you want to export the results back to the database
        # Settings:        
        'storage_extendable':False,   # state if you want storages to be installed at each node if necessary.
        'generator_noise':True,      # state if you want to apply a small generator noise 
        'reproduce_noise': False,    # state if you want to use a predefined set of random noise for the given.
                                     # scenario. if so, provide path, e.g. 'noise_values.csv'
        'minimize_loading':False,
        # Clustering:
        'k_mean_clustering': False,  # state if you want to perform a k-means clustering on the given network. 
                                     # State False or the value k (e.g. 20).
        'network_clustering': False, # state if you want to perform a clustering of HV buses to EHV buses.
        # Simplifications:
        'parallelisation':False,     # state if you want to run snapshots parallely.
        'skip_snapshots':False,
        'line_grouping': False,      # state if you want to group lines running between the same buses.
        'branch_capacity_factor': 0.7, # globally extend or lower branch capacities
        'load_shedding':False,       # meet the demand at very high cost; for debugging purposes.
        'comments':None }

In [ ]:
# Start eTraGo calculation with args setting
# create network object 
network = etrago(args_sh)

In [ ]:
# plot generation distripution
gen_dist(network, techs=None, snapshot=1,n_cols=3,gen_size=0.02)

In [ ]:
# plot stacked sum of nominal power for each generator type and timestep
plot_stacked_gen(network, resolution="MW")

In [ ]:
# plots
# make a line loading plot
fig,ax = plt.subplots(1,1)
fig.set_size_inches(12,10)
plot_line_loading(network)

In [ ]:
# plot to show extendable storages
storage_expansion(network)

In [ ]:
# curtailment per carrier / energy source
curtailment(network, carrier='wind_onshore')

In [ ]:
# delete network object
del network

# Using pf after lopf  Calculation of Germany and neighbours with 30 nodes  <a class="anchor" id="snapshot"></a>

<div style="text-align: right"> 
[top](#toc) </div>


In order to compute the grid losses we add an power flow calculation after our liniar opf calculation by setting *pf_post_lopf = True*. 



In [ ]:
args_lopf_pf = {# Setup and Configuration:
        'db': 'oedb',                # db session
        'gridversion': "v0.2.11",    # None for model_draft or Version number (e.g. v0.2.11) for grid schema
        'method': 'lopf',            # lopf or pf
        'pf_post_lopf': True,        # state whether you want to perform a pf after a lopf simulation
        'start_snapshot': 4393, 
        'end_snapshot' : 4417,
        'scn_name': 'NEP 2035',      # state which scenario you want to run: Status Quo, NEP 2035, eGo100
        'solver': 'glpk',            # glpk, cplex or gurobi
        # Export options:
        'lpfile': False,             # state if and where you want to save pyomo's lp file: False or /path/tofolder
        'results': False,            # state if and where you want to save results as csv: False or /path/tofolder
        'export': False,             # state if you want to export the results back to the database
        # Settings:        
        'storage_extendable':False,   # state if you want storages to be installed at each node if necessary.
        'generator_noise':True,      # state if you want to apply a small generator noise 
        'reproduce_noise': False,    # state if you want to use a predefined set of random noise for the given.
                                     # scenario. if so, provide path, e.g. 'noise_values.csv'
        'minimize_loading':False,
        # Clustering:
        'k_mean_clustering': 30,  # state if you want to perform a k-means clustering on the given network. 
                                     # State False or the value k (e.g. 20).
        'network_clustering': False, # state if you want to perform a clustering of HV buses to EHV buses.
        # Simplifications:
        'parallelisation':False,     # state if you want to run snapshots parallely.
        'skip_snapshots':False,
        'line_grouping': False,      # state if you want to group lines running between the same buses.
        'branch_capacity_factor': 0.7, # globally extend or lower branch capacities
        'load_shedding':False,       # meet the demand at very high cost; for debugging purposes.
        'comments':None }

In [ ]:
# Start eTraGo calculation with args setting
# create network object
network = etrago(args_lopf_pf)

In [ ]:
# plot stacked sum of nominal power for each generator type and timestep
plot_stacked_gen(network, resolution="MW")
#plt.close()

In [ ]:
# plots
# make a line loading plot
fig,ax = plt.subplots(1,1)
fig.set_size_inches(8,8)

plot_line_loading(network)

### Calculate grid losses

We are using the function [calc_line_losses(network)](http://etrago.readthedocs.io/en/latest/_modules/etrago/tools/utilities.html#calc_line_losses).

In [ ]:

# calcualte total grid losses
calc_line_losses(network)

### Plot line costs

In [ ]:
# Line losses
# calculate apparent power S = sqrt(p² + q²) [in MW]
s0_lines = ((network.lines_t.p0**2 + network.lines_t.q0**2).\
    apply(np.sqrt)) 

# calculate current I = S / U [in A]
i0_lines = np.multiply(s0_lines, 1000000) / np.multiply(network.lines.v_nom, 1000) 

# calculate losses per line and timestep network.lines_t.line_losses = I² * R [in MW]
network.lines_t.losses = np.divide(i0_lines**2 * network.lines.r, 1000000)

# calculate total losses per line [in MW]
network.lines = network.lines.assign(losses=np.sum(network.lines_t.losses).values)

# prepare plotting
timestep =1
cmap = plt.cm.jet

fig,ax = plt.subplots(1,1)
fig.set_size_inches(6,4)

# do the plotting
lc= network.plot(line_colors=network.lines.losses, line_cmap=cmap,
                  title="Line loading", line_widths=0.55)

cb = plt.colorbar(lc[1])
cb.set_label('Locational line losses in (EUR/MWh)')
plt.show()

### Plot the reactive power

In [ ]:
# get snapshot
now = network.snapshots[2]
#plot the reactive power
fig,ax = plt.subplots(1,1)

fig.set_size_inches(6,6)

q = network.buses_t.q.sum()#.loc[now]

bus_colors = pd.Series("r",network.buses.index)
bus_colors[q< 0.] = "b"


network.plot(bus_sizes=abs(q)*0.005,ax=ax,bus_colors=bus_colors,title="Reactive power feed-in (red=+ve, blue=-ve)")

## Have a  look into the Programm on Github

* <https://github.com/openego/eTraGo>